<a href="https://colab.research.google.com/github/juniormusasizi61/Post-Harvest-Loss-Prediction-Model-Building-/blob/main/deep_q_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Data Preparation & Feature Selection**


(This stage is shared across several models as they benefit from having only the most relevant inputs.)

Before diving into the model implementations, we must prepare our data. We first clean the data, encode categorical variables, and normalize numeric ones. Then we use Lasso (L1 regularization) to select a sparse set of features.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# --- Load the Dataset ---
# Read the CSV file into a pandas DataFrame.
data = pd.read_csv("/content/drive/MyDrive/post_harvest.csv")

# --- Handle Missing Values ---

# Define the numerical columns.
numerical_columns = ["loss_percentage_original", "cpc_code", "year"]

# Convert the numerical columns to numeric types.
# Any values that cannot be converted will be set to NaN.
data[numerical_columns] = data[numerical_columns].apply(pd.to_numeric, errors="coerce")

# Impute missing values in numerical columns with the mean of each column.
num_imputer = SimpleImputer(strategy="mean")
data[numerical_columns] = num_imputer.fit_transform(data[numerical_columns])

# Define the categorical columns.
categorical_columns = ["loss_percentage", "activity"]

# Impute missing values in categorical columns with the most frequent value.
cat_imputer = SimpleImputer(strategy="most_frequent")
data[categorical_columns] = cat_imputer.fit_transform(data[categorical_columns])

# --- Encode Categorical Variables ---

# # Initialize OneHotEncoder:
# - sparse_output=False returns a dense array.
# - drop="first" avoids the dummy variable trap by dropping the first category.
encoder = OneHotEncoder(sparse_output=False, drop="first")
encoded_categories = encoder.fit_transform(data[categorical_columns])

# Create a DataFrame from the encoded categories.
encoded_category_df = pd.DataFrame(encoded_categories, columns=encoder.get_feature_names_out())

# Remove the original categorical columns from the dataset.
data = data.drop(columns=categorical_columns)

# Append the encoded categorical DataFrame to the original DataFrame.
data = pd.concat([data, encoded_category_df], axis=1)

# Optionally, print the list of columns to verify the changes.
print(data.columns.tolist())

# --- Normalize Numerical Features ---

# Initialize StandardScaler to standardize numerical features.
scaler = StandardScaler()
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

# --- Save the Preprocessed Dataset ---

# Export the processed DataFrame to a new CSV file. which is compressed
data.to_csv("processed_post_harvest_data.csv", index=False, compression="gzip")
print("✅ Data Preprocessing Complete!")


['Unnamed: 0', 'm49_code', 'cpc_code', 'commodity', 'year', 'loss_percentage_original', 'loss_quantity', 'food_supply_stage', 'treatment', 'cause_of_loss', 'sample_size', 'method_data_collection', 'reference', 'url', 'notes', 'loss_percentage_0.00596777', 'loss_percentage_0.01', 'loss_percentage_0.0124215', 'loss_percentage_0.0138344', 'loss_percentage_0.02', 'loss_percentage_0.0232144', 'loss_percentage_0.0291458', 'loss_percentage_0.03', 'loss_percentage_0.0302437', 'loss_percentage_0.0336077', 'loss_percentage_0.033767', 'loss_percentage_0.0347729', 'loss_percentage_0.04', 'loss_percentage_0.05', 'loss_percentage_0.06', 'loss_percentage_0.07', 'loss_percentage_0.08', 'loss_percentage_0.0806246', 'loss_percentage_0.0819526', 'loss_percentage_0.0820745', 'loss_percentage_0.083543', 'loss_percentage_0.0847629', 'loss_percentage_0.0876717', 'loss_percentage_0.089166', 'loss_percentage_0.09', 'loss_percentage_0.0933903', 'loss_percentage_0.0958458', 'loss_percentage_0.0972093', 'loss_per

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
import pandas as pd

# Load the data with appropriate parameters to handle compression
data = pd.read_csv("/content/processed_post_harvest_data.csv",
                   compression="gzip",
                   low_memory=False)

# Separate features and target variables
# We exclude all columns that represent loss percentages from our features
X = data[[col for col in data.columns if not col.startswith('loss_percentage_')]]
y = data[[col for col in data.columns if col.startswith('loss_percentage_')]]

# Split the data into training and testing sets
# The random_state ensures reproducibility of results
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

# Convert categorical variables to numerical using one-hot encoding
X_train_encoded = pd.get_dummies(X_train, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, drop_first=True)

# Ensure consistent columns between train and test sets after encoding
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded,
                                                       join='inner',
                                                       axis=1)

# Create and fit the feature selector
# We increase max_iter to ensure convergence
selector = SelectFromModel(estimator=Lasso(alpha=0.1, max_iter=1000))

# Since y might have multiple columns (multiple loss percentage targets),
# we need to handle this appropriately
if y_train.shape[1] > 1:
    # If we have multiple targets, let's use the first one for feature selection
    # You might want to adjust this strategy based on your specific needs
    selector.fit(X_train_encoded, y_train.iloc[:, 0])
else:
    selector.fit(X_train_encoded, y_train)

# Get the selected feature mask and feature names
selected_features_mask = selector.get_support()
selected_features = X_train_encoded.columns[selected_features_mask].tolist()

# Print the selected features
print("Selected Features:", selected_features)
print(f"Number of selected features: {len(selected_features)}")

# Select only the important features for both training and test sets
X_train_selected = X_train_encoded[selected_features]
X_test_selected = X_test_encoded[selected_features]

# Save the processed datasets
X_train_selected.to_csv("X_train_selected.csv", index=False)
X_test_selected.to_csv("X_test_selected.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
y_test.to_csv("y_test.csv", index=False)

print("✅ Feature Selection Complete!")

Selected Features: ['Unnamed: 0', 'm49_code', 'food_supply_stage_Harvest']
Number of selected features: 3
✅ Feature Selection Complete!


Feature scaling
lets now scale our selected features to have zero mean and unit variance for neural network training
Now, let's prepare our data for the deep learning models. We need to scale our features properly:


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler

# Scale our selected features to have zero mean and unit variance
# This is crucial for neural network training
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

# Convert to PyTorch tensors for deep learning
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train.values)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test.values)

The first model shall be the teacher model, we shall build and train it. This will be a deep neural network that learns the complex patterns in our post-harvest loss data:

In [ ]:
class TeacherModel(nn.Module):
    def __init__(self, input_size):
        super(TeacherModel, self).__init__()

        # Create a deep architecture suitable for complex pattern recognition
        self.layers = nn.Sequential(
            # First layer - takes our selected features as input
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),  # Batch normalization for stable training
            nn.Dropout(0.3),      # Dropout for regularization

            # Hidden layers
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),

            # Output layer - predicts loss percentages
            nn.Linear(64, y_train.shape[1])
        )

    def forward(self, x):
        return self.layers(x)

# Initialize teacher model with our selected feature count
teacher_model = TeacherModel(X_train_selected.shape[1])

# Set up optimizer and loss function
teacher_optimizer = optim.Adam(teacher_model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Create a training function with proper monitoring
def train_teacher(model, epochs=100, batch_size=32):
    """
    Trains the teacher model with batch processing and progress monitoring
    """
    # Convert data to DataLoader for batch processing
    dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    training_losses = []

    for epoch in range(epochs):
        model.train()  # Set to training mode
        epoch_losses = []

        for batch_X, batch_y in dataloader:
            # Forward pass
            teacher_optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)

            # Backward pass and optimization
            loss.backward()
            teacher_optimizer.step()

            epoch_losses.append(loss.item())

        # Calculate average loss for the epoch
        avg_loss = np.mean(epoch_losses)
        training_losses.append(avg_loss)

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Average Loss: {avg_loss:.4f}')

            # Calculate validation metrics
            model.eval()
            with torch.no_grad():
                val_predictions = model(X_test_tensor)
                val_loss = criterion(val_predictions, y_test_tensor)
                print(f'Validation Loss: {val_loss.item():.4f}')

    return training_losses

# Train the teacher model
print("Training Teacher Model...")
teacher_losses = train_teacher(teacher_model)

Training Teacher Model...
Epoch 0, Average Loss: 0.0227
Validation Loss: 0.0012
Epoch 10, Average Loss: 0.0013
Validation Loss: 0.0010
Epoch 20, Average Loss: 0.0013
Validation Loss: 0.0010
Epoch 30, Average Loss: 0.0013
Validation Loss: 0.0010
Epoch 40, Average Loss: 0.0012
Validation Loss: 0.0010
Epoch 50, Average Loss: 0.0012
Validation Loss: 0.0010
Epoch 60, Average Loss: 0.0012
Validation Loss: 0.0010
Epoch 70, Average Loss: 0.0012
Validation Loss: 0.0010
Epoch 80, Average Loss: 0.0012
Validation Loss: 0.0010
Epoch 90, Average Loss: 0.0012
Validation Loss: 0.0010


Now let's create our Student Model, which will be simpler but will learn from the Teacher Model through knowledge distillation:

In [6]:
import torch.nn as nn
class StudentModel(nn.Module):
    def __init__(self, input_size):
        super(StudentModel, self).__init__()

        # Simpler architecture that will be enhanced through knowledge distillation
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),

            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),

            nn.Linear(32, y_train.shape[1])
        )

    def forward(self, x):
        return self.layers(x)

# Initialize the student model
student_model = StudentModel(X_train_selected.shape[1])

# Create the knowledge distillation loss function
class DistillationLoss(nn.Module):
    def __init__(self, temperature=3.0, alpha=0.7):
        super(DistillationLoss, self).__init__()
        self.temperature = temperature
        self.alpha = alpha
        self.criterion = nn.KLDivLoss(reduction='batchmean')
        self.mse = nn.MSELoss()

    def forward(self, student_outputs, teacher_outputs, targets):
        """
        Combines knowledge distillation loss with regular supervised loss
        """
        # Soften probability distributions
        soft_targets = nn.functional.softmax(teacher_outputs / self.temperature, dim=1)
        soft_predictions = nn.functional.log_softmax(student_outputs / self.temperature, dim=1)

        # Calculate distillation loss and regular loss
        distillation_loss = self.criterion(soft_predictions, soft_targets) * (self.temperature ** 2)
        student_loss = self.mse(student_outputs, targets)

        # Combine losses
        total_loss = (self.alpha * distillation_loss) + ((1 - self.alpha) * student_loss)
        return total_loss

# Set up student training
distillation_criterion = DistillationLoss()
student_optimizer = optim.Adam(student_model.parameters(), lr=0.001)

def train_student_with_distillation(student_model, teacher_model, epochs=100, batch_size=32):
    """
    Trains the student model using knowledge distillation from the teacher
    """
    dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    teacher_model.eval()  # Set teacher to evaluation mode
    training_losses = []

    for epoch in range(epochs):
        student_model.train()
        epoch_losses = []

        for batch_X, batch_y in dataloader:
            # Get teacher predictions
            with torch.no_grad():
                teacher_predictions = teacher_model(batch_X)

            # Train student
            student_optimizer.zero_grad()
            student_predictions = student_model(batch_X)

            # Calculate distillation loss
            loss = distillation_criterion(student_predictions, teacher_predictions, batch_y)

            loss.backward()
            student_optimizer.step()

            epoch_losses.append(loss.item())

        avg_loss = np.mean(epoch_losses)
        training_losses.append(avg_loss)

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Average Loss: {avg_loss:.4f}')

    return training_losses

# Train the student model
print("\nTraining Student Model with Knowledge Distillation...")
student_losses = train_student_with_distillation(student_model, teacher_model)


Training Student Model with Knowledge Distillation...
Epoch 0, Average Loss: 0.0191
Epoch 10, Average Loss: 0.0004
Epoch 20, Average Loss: 0.0004
Epoch 30, Average Loss: 0.0004
Epoch 40, Average Loss: 0.0004
Epoch 50, Average Loss: 0.0004
Epoch 60, Average Loss: 0.0004
Epoch 70, Average Loss: 0.0004
Epoch 80, Average Loss: 0.0004
Epoch 90, Average Loss: 0.0004


Knowledge Distillation framework

In [7]:
class DistillationLoss(nn.Module):
    def __init__(self, temperature=3.0):
        super(DistillationLoss, self).__init__()
        self.temperature = temperature
        self.criterion = nn.KLDivLoss(reduction='batchmean')

    def forward(self, student_outputs, teacher_outputs, targets):
        soft_targets = nn.functional.softmax(teacher_outputs / self.temperature, dim=1)
        student_log_softmax = nn.functional.log_softmax(student_outputs / self.temperature, dim=1)
        distillation_loss = self.criterion(student_log_softmax, soft_targets)
        student_loss = criterion(student_outputs, targets)
        return 0.7 * distillation_loss + 0.3 * student_loss

distillation_criterion = DistillationLoss()
student_optimizer = optim.Adam(student_model.parameters(), lr=0.001)

Deep q learning environment

In [18]:
!pip install stable-baselines3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [23]:
import gym
from gym import spaces
from stable_baselines3 import DQN
class PostHarvestEnv(gym.Env):
    def __init__(self, X, y):
        super(PostHarvestEnv, self).__init__()
        self.X = X
        self.y = y
        self.current_step = 0

        # Define action and observation spaces
        self.action_space = spaces.Discrete(10)  # 10 different prediction adjustments
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(X.shape[1],)
        )

    def step(self, action):
        # Implement environment dynamics
        prediction = self.student_model(self.X[self.current_step])
        adjustment = (action - 5) / 10.0  # Convert action to adjustment
        final_prediction = prediction + adjustment

        reward = -abs(final_prediction - self.y[self.current_step])

        self.current_step += 1
        done = self.current_step >= len(self.X)

        return self.X[self.current_step], reward, done, {}

    def reset(self):
        self.current_step = 0
        return self.X[self.current_step]

Reinforcement learning agent

In [25]:
pip install shimmy


In [26]:
env = PostHarvestEnv(X_train_scaled, y_train.values)
dqn_agent = DQN("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Ensamble model

In [27]:
class EnsembleModel:
    def __init__(self, models):
        self.models = models

    def predict(self, X):
        predictions = [model(X) for model in self.models]
        return torch.mean(torch.stack(predictions), dim=0)

ensemble = EnsembleModel([teacher_model, student_model])


Time series model


In [28]:
class TimeSeriesLSTM(nn.Module):
    def __init__(self, input_size):
        super(TimeSeriesLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, 64, batch_first=True)
        self.fc = nn.Linear(64, y_train.shape[1])

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])

ts_model = TimeSeriesLSTM(X_train_selected.shape[1])

Uncertainty-Aware Model

In [29]:
class BayesianNetwork(nn.Module):
    def __init__(self, input_size):
        super(BayesianNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, y_train.shape[1] * 2)  # Mean and variance

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        mean, var = torch.chunk(x, 2, dim=1)
        return mean, torch.exp(var)

bayesian_model = BayesianNetwork(X_train_selected.shape[1])

Transfer learning model

In [30]:
class TransferModel(nn.Module):
    def __init__(self, base_model):
        super(TransferModel, self).__init__()
        self.base = base_model
        self.adaptation = nn.Linear(y_train.shape[1], y_train.shape[1])

    def forward(self, x):
        x = self.base(x)
        return self.adaptation(x)

transfer_model = TransferModel(teacher_model)

Meta-learning model

In [31]:
class MetaLearner(nn.Module):
    def __init__(self, model_list):
        super(MetaLearner, self).__init__()
        self.models = nn.ModuleList(model_list)
        self.attention = nn.Linear(len(model_list), 1)

    def forward(self, x):
        predictions = [model(x) for model in self.models]
        stacked = torch.stack(predictions, dim=1)
        weights = torch.softmax(self.attention(stacked), dim=1)
        return torch.sum(weights * stacked, dim=1)

meta_learner = MetaLearner([
    teacher_model, student_model, ts_model, bayesian_model, transfer_model
])

Training and evaluation pipeline

In [32]:
def train_and_evaluate(models, X_train, y_train, X_test, y_test, epochs=100):
    results = {}
    for name, model in models.items():
        print(f"\nTraining {name}...")

        # Training
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        for epoch in range(epochs):
            optimizer.zero_grad()

            if isinstance(model, BayesianNetwork):
                mean, var = model(X_train_tensor)
                loss = gaussian_nll_loss(mean, var, y_train_tensor)
            else:
                outputs = model(X_train_tensor)
                loss = criterion(outputs, y_train_tensor)

            loss.backward()
            optimizer.step()

            if epoch % 10 == 0:
                print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

        # Evaluation
        model.eval()
        with torch.no_grad():
            if isinstance(model, BayesianNetwork):
                predictions, _ = model(X_test_tensor)
            else:
                predictions = model(X_test_tensor)

            mse = mean_squared_error(y_test, predictions.numpy())
            r2 = r2_score(y_test, predictions.numpy())

            results[name] = {'MSE': mse, 'R2': r2}

    return results

# Train and evaluate all models
models = {
    'Teacher': teacher_model,
    'Student': student_model,
    'Ensemble': ensemble,
    'TimeSeries': ts_model,
    'Bayesian': bayesian_model,
    'Transfer': transfer_model,
    'MetaLearner': meta_learner
}

results = train_and_evaluate(models, X_train_tensor, y_train_tensor,
                           X_test_tensor, y_test_tensor)

# Print results
for model_name, metrics in results.items():
    print(f"\n{model_name} Results:")
    print(f"MSE: {metrics['MSE']:.4f}")
    print(f"R2: {metrics['R2']:.4f}")


Training Teacher...
Epoch 0, Loss: 0.0012
Epoch 10, Loss: 0.0012
Epoch 20, Loss: 0.0012
Epoch 30, Loss: 0.0012
Epoch 40, Loss: 0.0012
Epoch 50, Loss: 0.0012
Epoch 60, Loss: 0.0012
Epoch 70, Loss: 0.0012
Epoch 80, Loss: 0.0012
Epoch 90, Loss: 0.0012


NameError: name 'mean_squared_error' is not defined